## 2. Just In Time Compilation with JAX

- Original: [JAX Tutorial: Just In Time Compilation with JAX](https://jax.readthedocs.io/en/latest/jax-101/02-jitting.html)
- Korean: [JAX 튜토리얼: JAX의 Just-In-Time 컴파일](https://jax-kr.readthedocs.io/ko/latest/JAX101/%ED%98%95%EC%84%AD%EC%B4%88%EB%B2%8C_Jitting_functions_in_JAX.html)

### JAX transform은 어떻게 동작하는가

- JAX를 사용하면 Python 함수를 변환할 수 있음

- 이러한 작업은, Python 함수를 'jaxpr'이라는 간단한 중간 언어로 변환해 수행되고 작동함

- `jax.makr_jaxpr`를 사용해 함수의 jaxpr 표현을 표시할 수 있음

In [2]:
import jax
import jax.numpy as jnp

In [3]:
global_list = []

def log2(x):
    global_list.append(x)
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2

print(jax.make_jaxpr(log2)(3.0))

{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


- 위 출력에 대한 자세한 의미는 '[Understanding Jaxprs](https://jax.readthedocs.io/en/latest/jaxpr.html)'에서 확인할 수 있음

- 중요한 점은, jaxpr가 함수의 side-effect를 포착하지 못한다는 점

    - `global_list.append(x)`에 상응하는 것이 없음

    - JAX가 side-effect가 없는, 즉 functionally-pure한 코드만을 이해하도록 설계되었기 때문에 (Details: [JAX - The Sharp Bits: Pure Functions](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#pure-functions))

- 물론 impure한 함수들은 작성될 수 있고 실행까지 될 수도 있지만, JAX가 jaxpr로 이를 변환한 이후에도 이것이 동작할 것이라 보장하지 않음

    - 그러나 JAX로 변환된 함수의 side-effect는 첫 번째 호출 시 딱 한 번만 실행되고 다시 발생하지 않을 것으로 예상할 수 있음 (여기에 의존해서는 안되지만)

    - 이는 JAX가 '추적 (tracing)'이라는 프로세스를 통해 jaxpr을 생성하기 때문

- Tracing을 수행할 때, JAX는 각 인수를 `tracer` 객체로 wrapping함

    - 그 다음 `tracer`는 함수 호출 도중 수행된 모든 JAX 연산을 기록하고, 이 기록을 사용해 전체 함수를 재구성함

    - 이 재구성된 출력이 바로 jaxpr인 것

    - `tracer`가 파이썬의 side-effect를 기록하지 않기 때문에 jaxpr에 나타나지 않는 것이지만, 추적 과정 자체에서는 여전히 side-effect가 발생함

- 참고: 파이썬의 `print()`는 functionally-pure하지 않아, 모든 `print()`문의 호출은 추적 과정 중에만 발생하며 jaxpr에는 나타나지 않음

In [4]:
def log2_with_print(x):
    print("printed x:", x)
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2

print(jax.make_jaxpr(log2_with_print)(3.))

printed x: Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>
{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


- 위 출력에서 `x`가 어떻게 `Traced` 객체로 출력되는지 확인할 수 있음

- 파이썬 코드가 한 번 이상 실행된다는 사실은 어디까지나 구현상의 세부 사항이므로 이에 의존해서는 안됨

    - 그래도 디버깅 시 계산 중간 값 출력에 사용할 수는 있으므로 이해하고 있으면 유용함

- jaxpr은 주어진 매개변수에서 실행되는 함수만을 캡처한다는 것을 이해해야 함

    - 예를 들어, 조건문이 있는 경우 jaxpr은 매개변수에 해당하는 조건문 브랜치에 대해서만 쓰여짐

In [7]:
def log2_if_rank_2(x):
    if x.ndim == 2:
        ln_x = jnp.log(x)
        ln_2 = jnp.log(2.0)
        return ln_x / ln_2
    else:
        return x
    
print(jax.make_jaxpr(log2_if_rank_2)(jax.numpy.array([1, 2, 3])))

{ lambda ; a:i32[3]. let  in (a,) }


### JIT을 통한 함수 컴파일링

- 앞서 설명되었듯이, JAX는 동일한 코드에서 CPU/GPU/TPU 등 다양한 백엔드 내 연산을 수행할 수 있음

In [8]:
def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(1000000)
%timeit selu(x).block_until_ready()

2.39 ms ± 342 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


- 위 코드는 한 번에 하나의 연산만을 accelerator로 보내기 때문에, XLA 컴파일러의 함수 최적화 기능을 제한하고 있음

- 당연히 XLA 컴파일러에 가능한 한 많은 코드를 제공해 전체를 완전히 최적화하는 것이 중요함

- JAX는 이를 위해 `jax.jit` 변환을 제공하며, JAX 호환 함수를 JIT 컴파일링 함

In [9]:
selu_jit = jax.jit(selu)
selu_jit(x).block_until_ready() # warm up
%timeit selu_jit(x).block_until_ready()

218 µs ± 8.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


- 여기서 `selu_jit`을 `selu`의 jit 컴파일링 버전으로 정의했으며, 해당 컴파일 과정 내에서 발생하는 일은 다음과 같음

    1. JAX는 tracing을 수행 (`tracer`로 wrapping할 입력이 필요하기 때문)

    2. jaxpr이 XLA를 사용해 GPU/TPU에 최적화된 매우 효율적인 코드로 컴파일

    3. 컴파일된 코드가 호출을 만족시키기 위해 실행

    - 이후 `selu_jit`에 대한 호출은 컴파일된 코드를 직접 사용해 파이썬 구현을 완전히 건너뛰게 됨

- 참고: `.block_until_ready()`를 통해 warm up을 해준 이유는 단순히 공정한 실행 시간 측정을 위한 것임. 해당 메서드는 동기화 실행을 통해 컴파일 시간을 벤치마크에서 제외시켜 줌.



### 그냥 다 JIT 컴파일링 해버리면 안 되나?

- 모든 함수에 `jax.jit`을 적용할 수 없는지에 대한 의문

- 적용해야할 때와 적용하지 말아야 할 때를 이해하기 위해, 먼저 JIT 컴파일링이 작동하지 않는 몇 가지 경우를 확인할 수 있음

In [12]:
# x의 값에 조건이 붙는 경우

def f(x):
    if x > 0:
        return x
    else:
        return 2 * x
    
f_jit = jax.jit(f)
print(f_jit(3))

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[]..
The error occurred while tracing the function f at /tmp/ipykernel_3133527/3232586665.py:1 for jit. This concrete value was not available in Python because it depends on the value of the argument x.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerBoolConversionError

In [13]:
# x와 n의 값에 조건이 붙는 while 반복문

def g(x, n):
    i = 0
    while i < n:
        i += 1
    return x + i

g_jit = jax.jit(g)
print(g_jit(10, 20))

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[]..
The error occurred while tracing the function g at /tmp/ipykernel_3133527/834857431.py:3 for jit. This concrete value was not available in Python because it depends on the value of the argument n.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerBoolConversionError

- JIT 컴파일링 하려는 함수에 대한 입력 값에 조건을 걸려고 했다는 것이 문제

    - jaxpr이 tracing에 사용되는 실제 값에 의존하기 때문

- Tracing에 사용하는 값에 대한 정보가 구체적일수록 표준 파이썬 control flow를 사용해 더 많이 표현(express)할 수 있음

    - 그러나, '너무 구체적'인 경우 다른 값에 대해 동일한 tracing 함수를 재사용할 수 없다는 뜻

    - JAX는 다양한 목적에 따라, 다양한 추상화 수준에서 이를 tracing함으로써 해결

- `jax.jit`의 기본 레벨은 `ShapedArray`로, 각 tracer에는 구체적인 모양은 있지만 구체적인 값은 없음

    - 이렇게 하면 컴파일된 함수가 ML에서 자주 사용되는 것 처럼 동일한 shape의 모든 가능한 입력에 대해 작동할 수 있음

    - 그러나, tracer에는 구체적인 값이 없기 때문에 하나의 값을 조건으로 지정하려고 하는 경우 위와 같은 오류 발생

- `jax.grad`에서는 제약 조건이 조금 더 완화되어 더 많은 작업을 수행할 수 있음

    - 그러나 여러 변환을 구성하는 경우 가장 엄격한 변환 제약 조건을 충족해야 함

    - 따라서, `jit(grad(f))`의 경우 `f`는 값에 따른 조건이 지정되지 않아야 함

    - Python control flow와 JAX 간의 상호 작용에 대한 자세한 내용은 [JAX - The Shapr Bits: Control Flow](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#control-flow) 참조

- 해결 방안

    - 이러한 문제를 해결하는 하나의 방법은, 값에 조건문을 사용하지 않도록 코드를 다시 작성하는 것

    - 또는 `jax.lax.cond`와 같은 special control flow operators`를 사용하는 것이지만, 이것이 불가능한 경우도 많음

        - 이러한 경우 함수의 일부만 jitting하는 것을 고려할 수 있음

        - 예를 들어, 가장 계산 비용이 많이 드는 부분이 반복문 내에 존재하는 경우 해당 내부 부분만 JIT할 수 있음 (이러한 경우 캐싱 확인 필수, 다음 섹션에서 설명)

In [15]:
# 내부가 JIT 컴파일된, x와 n의 값에 조건이 붙는 while 반복문

@jax.jit
def loop_body(prev_i):
    return prev_i + 1

def g_inner_jitted(x, n):
    i = 0
    while i < n:
        i = loop_body(i)
    return x + i

display(g_inner_jitted(10, 20))

Array(30, dtype=int32, weak_type=True)

- 입력 값에 조건이 있는 함수를 JIT해야 하는 경우, `static_argnums` 또는 `static_argnames`를 지정해 특정 입력에 대해 조금 덜 abstract한 tracer를 사용하도록 JAX에 지시할 수 있음

- 그 결과 jaxpr의 유연성은 떨어지기 때문에 지정된 static input의 모든 새 값에 대해 JAX가 함수를 재컴파일해야만 함

- 즉, 함수가 제한된 다른 값을 가져오는 것이 보장되는 경우에 사용하기 좋은 전략이라는 것

In [17]:
f_jit_correct = jax.jit(f, static_argnums=0)
print(f_jit_correct(10))

10


In [18]:
g_jit_correct = jax.jit(g, static_argnames=['n'])
print(g_jit_correct(10, 20))

30


- `@jax.jit`과 같이 데코레이터를 통해 사용할 때 argument를 지정하려면 일반적으로 사용하는 파이썬 패턴은 `functools.partial`

In [19]:
from functools import partial

@partial(jax.jit, static_argnames=['n'])
def g_jit_decorated(x, n):
    i = 0
    while i < n:
        i += 1
    return x + i

print(g_jit_decorated(10, 20))

30


### JIT은 언제 사용하는가

- 위 많은 예시들 중 대부분은 jitting이 크게 의미가 없고 좋지 않음

In [20]:
print("g-jitted:")
%timeit g_jit_correct(10, 20).block_until_ready()

print("g:")
%timeit g(10, 20)

g-jitted:
4.42 µs ± 63.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
g:
280 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


- 이는 `jax.jit` 자체에 약간의 오버헤드가 있기 때문

    - 따라서 일반적으로 컴파일된 함수가 복잡하고, 여러번 실행해야 하는 경우에만 시간을 절약할 수 있음

    - 다행히도, 이는 크고 복잡한 모델을 컴파일한 다음 수백만 번의 반복을 통해 실행하는 ML/DL에서 흔히 볼 수 있는 현상임

- 일반적으로 연산 중 가능한 가장 큰 chunk, 즉 전체 업데이트 단계를 JIT 컴파일링하는 것이 이상적이며, 이렇게 하는 것이 컴파일러가 최대한 자유롭게 최적화할 수 있는 방법

### 캐싱

- `f = jax.jit(g)`를 정의한다고 가정

    - 처음 `f`를 호출하면 컴파일되고 그 결과 XLA 코드가 캐싱됨

    - 이후 `f`를 호출하면 캐싱된 코드가 재사용됨 --> `jax.jit`이 컴파일에 드는 초기 비용을 보상하는 방식

- `static_argnums`를 지정하면 캐시된 코드는 static으로 레이블이 지정된 argument의 동일한 값에 대해서만 사용됨

    - 이 중 하나라도 변경되는 경우 재컴파일링이 발생하기 때문에, 값이 많으면 연산보다 컴파일링에 더 많은 시간이 소요될 수 있음

- 반복문 루프 내에서 `jax.jit`을 호출하지 말 것

    - 대부분의 경우 JAX는 컴파일되고 캐시된 함수를 이후 JIT 호출에서 사용할 수 있음

    - 그러나 캐시는 함수의 hash에 의존하기 때문에 동등한(equivalent) 함수가 재정의 될 때 문제가 발생할 수 있으며, 이로 인해 반복문 내에서 매번 불필요한 컴파일이 발생

In [21]:
def unjitted_loop_body(prev_i):
    return prev_i + 1

In [22]:
def g_inner_jitted_partial(x, n):
    i = 0
    while i < n:
        # 이렇게 하면 안됨
        # partial이 매 반복마다 함수에 대한 새로운 hash를 생성하기 때문에, 매번 컴파일링이 들어가게 됨
        i = jax.jit(partial(unjitted_loop_body))(i)
    return x + i

def g_inner_jitted_lambda(x, n):
    i = 0
    while i < n:
        # 이렇게 하면 안됨
        # lambda도 마찬가지로 매 반복마다 함수에 대한 새로운 hash를 생성하기 때문에, 매번 컴파일링이 들어가게 됨
        i = jax.jit(lambda x: unjitted_loop_body(x))(i)
    return x + i

def g_inner_jitted_normal(x, n):
    i = 0
    while i < n:
        # 이건 괜찮음
        # JAX가 캐싱된, 컴파일된 함수를 찾을 수 있음
        i = jax.jit(unjitted_loop_body)(i)
    return x + i

In [23]:
print("jit called in a loop with partials:")
%timeit g_inner_jitted_partial(10, 20).block_until_ready()

print("jit called in a loop with lambdas:")
%timeit g_inner_jitted_lambda(10, 20).block_until_ready()

print("jit called in a loop with caching:")
%timeit g_inner_jitted_normal(10, 20).block_until_ready()

jit called in a loop with partials:
298 ms ± 9.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
jit called in a loop with lambdas:
358 ms ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
jit called in a loop with caching:
1.38 ms ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
